In [2]:
import xml.dom
import xml.dom.minidom
import os
from PIL import Image

In [2]:
def create_xml_file(folder, filename, width, height, depth, objectList):
    doc = xml.dom.minidom.Document()
    root = doc.createElement('annotation')
    doc.appendChild(root)

    nodeFolder = doc.createElement('folder')
    nodeFileName = doc.createElement('filename')
    nodeFolder.appendChild(doc.createTextNode(folder))
    nodeFileName.appendChild(doc.createTextNode(filename))
    root.appendChild(nodeFolder)
    root.appendChild(nodeFileName)

    nodeSize = doc.createElement('size')
    nodeWidth = doc.createElement('width')
    nodeHeight = doc.createElement('height')
    nodeDepth = doc.createElement('depth')
    nodeWidth.appendChild(doc.createTextNode(str(width)))
    nodeHeight.appendChild(doc.createTextNode(str(height)))
    nodeDepth.appendChild(doc.createTextNode(str(depth)))
    nodeSize.appendChild(nodeWidth)
    nodeSize.appendChild(nodeHeight)
    nodeSize.appendChild(nodeDepth)
    root.appendChild(nodeSize)

    nodeSegmented = doc.createElement('segmented')
    nodeSegmented.appendChild(doc.createTextNode('0'))
    root.appendChild(nodeSegmented)

    for obj in objectList:
        nodeObject = doc.createElement('object')
        nodeName = doc.createElement('name')
        nodePose = doc.createElement('pose')
        nodeTruncated = doc.createElement('truncated')
        nodeDifficult = doc.createElement('difficult')
        nodeBndbox = doc.createElement('bndbox')
        nodeXmin = doc.createElement('xmin')
        nodeYmin = doc.createElement('ymin')
        nodeXmax = doc.createElement('xmax')
        nodeYmax = doc.createElement('ymax')

        nodeName.appendChild(doc.createTextNode(obj['name']))
        nodePose.appendChild(doc.createTextNode("Unspecified"))
        nodeTruncated.appendChild(doc.createTextNode("0"))
        nodeDifficult.appendChild(doc.createTextNode("0"))
        nodeXmin.appendChild(doc.createTextNode(str(obj['xmin'])))
        nodeYmin.appendChild(doc.createTextNode(str(obj['ymin'])))
        nodeXmax.appendChild(doc.createTextNode(str(obj['xmax'])))
        nodeYmax.appendChild(doc.createTextNode(str(obj['ymax'])))

        nodeBndbox.appendChild(nodeXmin)
        nodeBndbox.appendChild(nodeYmin)
        nodeBndbox.appendChild(nodeXmax)
        nodeBndbox.appendChild(nodeYmax)
        nodeObject.appendChild(nodeName)
        nodeObject.appendChild(nodePose)
        nodeObject.appendChild(nodeTruncated)
        nodeObject.appendChild(nodeDifficult)
        nodeObject.appendChild(nodeBndbox)

        root.appendChild(nodeObject)
    
    return doc

In [3]:
def parse_bounding_box(dir):
    with open(dir, 'r') as f:
        line = f.readline().strip().split(" ")
        x1 = round(float(line[0]))
        y1 = round(float(line[1]))
        x2 = round(float(line[2]))
        y2 = round(float(line[3]))
    return (x1,y1,x2,y2)

In [13]:
def create_emotion_annotation(label_list, bbox_dir, img_dir, anno_dir):
    label_mapping = {"1":"Surprise", "2":"Fear", "3":"Disgust", "4":"Happiness", "5":"Sadness",
                     "6":"Anger", "7":"Neutral"}

    with open(label_list, 'r') as f:
        for line in f:
            line = line.strip().split(" ")
            img_name = line[0]
            prefix = img_name.replace(".jpg", "")
            cls = label_mapping[line[1]]
            bbox = parse_bounding_box(bbox_dir+prefix+"_boundingbox.txt")
            folder = "Emotion"
            img = Image.open(os.path.join(img_dir, img_name))
            width, height = img.size
            depth = "3"
            obj = {"name":cls, "xmin":bbox[0], "ymin":bbox[1],
                   "xmax":bbox[2], "ymax":bbox[3]}
            
            doc = create_xml_file(folder, img_name, width, height, depth, [obj])
            fp = open(os.path.join(anno_dir, prefix+".xml"), 'w')
            doc.writexml(fp, indent='\t', addindent='\t', newl='\n', encoding="utf-8")

In [4]:
def create_imageset(label_list, imgset_dir, train_num=10000):
    ftest = open(os.path.join(imgset_dir, "test.txt"), 'w')
    ftrain = open(os.path.join(imgset_dir, "train.txt"), 'w')
    ftrainval = open(os.path.join(imgset_dir, "trainval.txt"), 'w')
    fval = open(os.path.join(imgset_dir, "val.txt"), 'w')
    
    ignore = {"train_00613", "train_01987", "train_04655", "train_05643", "train_06645", "train_08621", "train_08780", "train_10621" ,"test_1838"}
    with open(label_list, 'r') as f:
        for line in f:
            line = line.strip().split(" ")
            img_name = line[0].replace(".jpg", "")
            if img_name in ignore:
                continue
            if img_name.startswith("train"):
                ftrain.write(img_name+"\n")
                ftrainval.write(img_name+"\n")
            else:
                ftrainval.write(img_name+"\n")
                fval.write(img_name+"\n")
                ftest.write(img_name+"\n")
    ftest.close()
    ftrain.close()
    ftrainval.close()
    fval.close()

In [3]:
root = 'D:\\Courses\\2021 Semester 2\\Research Project'
label_list = os.path.join(root, "basic/EmoLabel/list_patition_label.txt")
bbox_dir = os.path.join(root, "basic/Annotation/boundingbox/")


anno_dir = os.path.join(root, "FasterRCNN/datasets/Emotion/Annotations")
img_dir = os.path.join(root, "FasterRCNN/datasets/Emotion/JPEGImages")
imgset_dir = os.path.join(root, "FasterRCNN/datasets/Emotion/ImageSets/Main")

In [9]:
create_emotion_annotation(label_list, bbox_dir, img_dir, anno_dir)

In [5]:
create_imageset(label_list, imgset_dir)

In [ ]:
# dataset: 0:gender, 1:race, 2:age
def create_annotation(label_dir, bbox_dir, img_dir, anno_dir, dataset):
    label_mapping = [{"0": "male", "1":"female", "2": "unsure"},{"0": "Caucasian", "1": "African-American", "2": "Asian"},{"0": "0-3", "1": "4-19", "2": "20-39", "3": "40-69", "4": "70+"}]
    
    for filename in os.listdir(label_dir):
        with open(os.path.join(label_dir, filename), 'r') as f:
            for i in range(5+dataset):
                f.readline()
            cls = label_mapping[dataset][f.readline().strip()]
            prefix = filename.replace("_manu_attri.txt", "")
            img_name = prefix + ".jpg"
            folder = "Race"
            bbox = parse_bounding_box(bbox_dir+prefix+"_boundingbox.txt")
            img = Image.open(os.path.join(img_dir, img_name))
            width, height = img.size
            depth = "3"
            obj = {"name":cls, "xmin":bbox[0], "ymin":bbox[1],
                   "xmax":bbox[2], "ymax":bbox[3]}
            doc = create_xml_file(folder, img_name, width, height, depth, [obj])
            fp = open(os.path.join(anno_dir, prefix+".xml"), 'w')
            doc.writexml(fp, indent='\t', addindent='\t', newl='\n', encoding="utf-8")

In [ ]:
root = 'D:\\Courses\\2021 Semester 2\\Research Project'
label_dir = os.path.join(root, "basic/Annotation/manual/")
bbox_dir = os.path.join(root, "basic/Annotation/boundingbox/")
img_dir = os.path.join(root, "FasterRCNN/datasets/Emotion/JPEGImages")

anno_dir = os.path.join(root, "FasterRCNN/datasets/Race/Annotations")

In [ ]:
create_annotation(label_dir, bbox_dir, img_dir, anno_dir, dataset)